# Coursera Applied Data Science Capstone
This notebook is intended for work toward the completion of the capstone project "The Battle of the Neighborhoods"

In [1]:
# Import needed libraries
import pandas as pd
import numpy as np

In [2]:
# Print statement for completion of first assignment
print("Hello Capstone Project Course!")

Hello Capstone Project Course!
